In [72]:
from typing import Optional, Iterable

import pandas as pd
import numpy as np

from time_series_anomaly_detection.abstractions import (
    TimeSeriesAnomalyDetector
)

import tensorflow as tf
from tensorflow.keras.layers import *

# this is xavier initializer as described in the paper
from tensorflow.keras.initializers import glorot_normal
from tensorflow.keras import Model
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator 
from tensorflow_probability.python.distributions import MultivariateNormalDiag

from scipy.stats import multivariate_normal
from sklearn.preprocessing import MinMaxScaler

In [73]:
class MultipleTimeseriesGenerator(Sequence):    
    def __init__(self, df_list, anomaly_list=None, time_window=8, shuffle=False, batch_size=32):
        super().__init__()
        if anomaly_list is None:
            self.anomaly_list = [np.zeros(len(series)) for series in df_list]
        else:
            self.anomaly_list = anomaly_list
            
        self.batch_size = batch_size
        self.generators = [TimeseriesGenerator(np.array(df_list[i]),np.array(self.anomaly_list[i]),length=time_window, 
                                               batch_size=batch_size, shuffle=shuffle) for i in range(len(df_list))]
        
        self.generator_lengths = [len(g) for g in self.generators]
        self.generator_indexes = np.cumsum(self.generator_lengths)
        self.len = np.sum(self.generator_lengths)

    def __len__(self):
        return self.len

    def __getitem__(self, index):
        # which series contains this index
        time_series_index = np.where(self.generator_indexes>index)[0][0]
        
        # get generator for the series, calculate position within than series and get its element
        element = self.generators[time_series_index][index % self.generator_indexes[max(0,time_series_index-1)]]
        return element

class SCVAEDetector(TimeSeriesAnomalyDetector):
    """
    Anomaly detector implemented as described in https://arxiv.org/pdf/1712.06343.pdf

    Parameters
    ----------
    id_columns: Iterable[str], optional
        ID columns used to identify individual time series.

        Should be specified in case the detector is provided with
        time series during training or inference with ID columns
        included. Using these columns the detector can separate individual
        time series and not use ID columns as feature columns.
        In case they are not specified, all columns are regarded as feature
        columns and the provided data is regarded as a single time series.
    """

    def __init__(
        self,
        latent_dim: Optional[int] = 50,
        time_window: Optional[int] = 8,
        batch_size: Optional[int] = 32,
        id_columns: Optional[Iterable[str]] = None
    ):
        super().__init__()
        self.latent_dim = latent_dim
        self.time_window = time_window
        self._id_columns = id_columns
        self.batch_size = batch_size
        
        
    def split_multiple_timeseries_by_id(self, df_series):
        return [pd.DataFrame(y).drop(self._id_columns,axis=1)
                for x, y in df_series.groupby(self._id_columns, as_index=False)]

    def predict_anomaly_scores(
        self, X: pd.DataFrame, *args, **kwargs
    ) -> pd.Series:
        # TODO: return predicted anomaly scores for the given samples
        pass

    def fit(self, X: pd.DataFrame, *args, **kwargs) -> None:
        self.feature_count = len(X.columns) - len(self._id_columns)
        self.model = self._build_model()
        self.scaler = MinMaxScaler()
        X = self.split_multiple_timeseries_by_id(X)
        self.scaler.fit(pd.concat(X))
        for i in range(len(X)):
            X[i][X[i].columns] = self.scaler.transform(X[i][X[i].columns])
        pass
    
    def _reparametrization_latent(self, args):
        mean, logvar = args
        # Adding Gaussian noise, avoiding backpropagation path
        eps = tf.random.normal(shape = (self.batch_size, self.latent_dim))
        return eps * tf.exp(logvar * 0.5) + mean
    
    def _reparametrization_series(self, args):
        mean, logvar = args
        eps = tf.random.normal(shape = (self.batch_size,self.time_window * self.feature_count))
        return eps * tf.exp(logvar * 0.5) + mean
    
    def _kl_loss(self, mean, logvar):
        return -0.5*tf.keras.backend.sum(1 + logvar - mean**2 - tf.exp(logvar),axis=-1)
    
    def _reconstruction_loss(self, input,output):
        return tf.keras.losses.BinaryCrossentropy(input, output=tf.keras.losses.Reduction.SUM)
    
    def _elbo_loss(self, input,output,mean_encoder,logvar_encoder,mean_decoder,logvar_decoder):
        kl = self._kl_loss(mean_encoder,logvar_encoder) + self._kl_loss(mean_decoder,logvar_decoder) 
        reconstruction = self._reconstruction_loss(input,output)
        return kl + reconstruction
    
    # https://github.com/Michedev/VAE_anomaly_detection/blob/master/VAE.py
    def _reconstruction_probability(self, X, mean, scale):
        p_l = multivariate_normal.pdf(X, mean=mean, cov=np.diag(scale))
        return reconstructed_prob
    
    def _build_model(self):        
        # Encoder
        input = Input(shape=(self.time_window,self.feature_count),batch_size=self.batch_size)
        
        # Squeeze convolution
        x = Conv1D(16, kernel_size=1, strides=1, kernel_initializer=glorot_normal, padding='same')(input)
        x = Activation('relu')(x)
        x = BatchNormalization(momentum=0.9)(x)

        # Extend (expand) convolutions
        extend1 = Conv1D(16, kernel_size=1, strides=1, kernel_initializer=glorot_normal, padding='same')(x)
        extend1 = Activation('relu')(extend1)
        extend1 = BatchNormalization(momentum=0.9)(extend1)
        extend2 = Conv1D(32, kernel_size=3, strides=1, kernel_initializer=glorot_normal, padding='same')(x)
        extend2 = Activation('relu')(extend2)
        extend2 = BatchNormalization(momentum=0.9)(extend2)
        x = Concatenate()([extend1, extend2])
    
        # Fully connected layers    
        x = Flatten()(x)
        mean_encoder = Dense(self.latent_dim)(x)
        logvar_encoder = Dense(self.latent_dim)(x)
        
        # "Sampling" using reparametrization trick        
        x = Lambda(self._reparametrization_latent, output_shape=(self.latent_dim,))([mean_encoder, logvar_encoder])
        x = Reshape((self.latent_dim,1))(x)
        # Decoder
        
        # Squeeze convolution
        x = Conv1DTranspose(16, kernel_size=1, strides=1, kernel_initializer=glorot_normal, padding='same')(x)
        x = Activation('relu')(x)
        x = BatchNormalization(momentum=0.9)(x)
    
        # Extend (expand) convolutions
        extend1 = Conv1DTranspose(16, kernel_size=1, strides=1, kernel_initializer=glorot_normal, padding='same')(x)
        extend1 = Activation('relu')(extend1)
        extend1 = BatchNormalization(momentum=0.9)(extend1)
        extend2 = Conv1DTranspose(1, kernel_size=3, strides=1, kernel_initializer=glorot_normal, padding='same')(x)
        extend2 = Activation('relu')(extend2) 
        extend2 = BatchNormalization(momentum=0.9)(extend2)        
        
        x = Concatenate()([extend1, extend2])        
        
        x = Flatten()(x)
        # Fully connected layers    
        mean_decoder = Dense(self.time_window * self.feature_count)(x)
        logvar_decoder = Dense(self.time_window * self.feature_count)(x)
        
        # "Sampling" using reparametrization trick        
        x = Lambda(self._reparametrization_series, output_shape=(self.time_window * self.feature_count,1))([mean_decoder, logvar_decoder])                
        output = Reshape((self.time_window, self.feature_count,))(x)
        
        return Model(inputs=input, outputs=[output,mean_encoder,logvar_encoder,mean_decoder,logvar_decoder])

In [74]:
def load_skab_valve1():
    all_series = []
    for file_num in range(16):
        df = pd.read_csv(f'../datasets/skab/valve1/{file_num}.csv',sep=';').drop(columns=['datetime', 'changepoint'])
        df['id_1'] = file_num
        df.pop('anomaly')
        all_series.append(df)    
    id_cols = ['id_1']    
    return pd.concat(all_series), id_cols

data,ids = load_skab_valve1()

scvae = SCVAEDetector(latent_dim=50,time_window=8,batch_size=32,id_columns=ids)
scvae.fit(data)